In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

'''
Reusing code:
https://stackabuse.com/k-nearest-neighbors-algorithm-in-python-and-scikit-learn/

Explaining result:
https://datascience.stackexchange.com/questions/65839/macro-average-and-weighted-average-meaning-in-classification-report


Test_fearMat_Clean: 4168
    Before edit: 4231 rows × 34 columns
    After edit: 4168 rows × 34 columns
    
    #removed 6 to 7
    total = 3749         (20% = 750; 80% = 2,999)
    

Train_fearMat_Clean: 16680
    Before edit: 16927 rows × 34 columns
    After edit: 16680 rows × 34 columns
    
    #removed 6 to 7
    total = 15045         (20% = 3,009; 80% = 12,036)

'''

import os
print(os.listdir(r"C:\Users\eddie\OneDrive\Desktop\CISC 4900"))         #print out what's in the file/directory

train = pd.read_csv(r"C:\Users\eddie\OneDrive\Desktop\CISC 4900\Test_dataset.csv")         #using test


#test = pd.read_csv(r"C:\Users\eddie\OneDrive\Desktop\CISC 4900\Train_dataset.csv")

['dataset_P1.csv', 'dataset_P2.csv', 'featMat.csv', 'featMatVersion2_5.csv', 'features', 'Old dataset', 'Test_dataset.csv', 'Touchalytics-ProjectDef.pdf', 'Train_dataset.csv']


In [2]:
#print
train.head()

,User_ID,Doc_ID,Inter-Stroke time,Stroke Duration,Start X,Start Y,Stop X,Sop Y,Direct end-to-end Distance,Mean resulant length,...,Average Direction,Length of Trajectory,Ratio dist and length of trajectory,Average velocity,Median acceleration at first 5 pts,Mid-Stroke pressure,Mid-Stroke area covered,Mid-Stroke finger orientation,Change of finger orientation,Phone orientation
0,35,5,0.608,0.426,31.6490,-36.588,18.5460,-41.124,13.8660,0.54406,...,3.036000,14.7170,0.94219,34.546,-64.736,0.35,0.044444,0.0,0,1
1,20,5,0.369,0.152,25.0730,-43.496,4.2515,-41.207,20.9470,0.50237,...,2.938600,20.9700,0.99889,137.960,-2642.200,0.49,0.155560,0.0,0,1
2,28,5,2.071,0.068,23.9830,-28.670,30.6330,-28.016,6.6819,0.99732,...,0.096037,6.6877,0.99912,98.349,-4700.000,0.62,0.144440,0.0,0,1
3,10,5,0.407,0.101,9.1722,-45.256,22.4770,-42.233,13.6440,0.99216,...,0.184680,13.6630,0.99860,135.280,-1673.300,0.73,0.044444,0.0,0,1
4,35,5,0.281,0.131,29.0290,-29.835,21.8720,-32.254,7.5541,0.67764,...,-2.727900,7.5586,0.99941,57.699,-845.600,0.48,0.044444,0.0,0,1


In [3]:
#print out the columns
train.columns

Index(['User_ID', 'Doc_ID', 'Inter-Stroke time', 'Stroke Duration', 'Start X',
       'Start Y', 'Stop X', 'Sop Y', 'Direct end-to-end Distance',
       'Mean resulant length', 'WSAD Flag', 'Direction of end-to-end Time',
       'Phone_ID', '20 per pairwise velocity', '50 per pairwise velocity',
       '80 per pairwise velocity', '20 per pairwise acc',
       '50 per pairwise acc', '80 per pairwise acc',
       'Median velocity at last 3pts',
       'Largest deviation from end-to-end line', '20 per dev line',
       '50 per dev line', '80 per dev line', 'Average Direction',
       'Length of Trajectory', 'Ratio dist and length of trajectory',
       'Average velocity', 'Median acceleration at first 5 pts',
       'Mid-Stroke pressure', 'Mid-Stroke area covered',
       'Mid-Stroke finger orientation', 'Change of finger orientation',
       'Phone orientation'],
      dtype='object')

In [4]:
#prints the amount of users in 'User_ID'
train['User_ID'].value_counts()

2     209
35    179
23    172
38    143
33    137
17    134
15    130
14    127
7     121
34    115
36    114
21    114
3     107
8      98
6      98
18     96
27     92
37     88
39     87
28     83
24     81
29     81
25     79
12     75
31     74
22     74
20     73
11     70
1      63
5      62
9      61
32     60
16     58
40     55
10     53
13     53
19     53
4      50
41     46
26     44
30     40
Name: User_ID, dtype: int64

In [5]:
#Choosing the columns for X
#Not using 'User_ID', 'Doc_ID', or 'Phone_ID'
X = train[['Inter-Stroke time', 'Stroke Duration', 'Start X', 'Start Y', 'Stop X', 'Sop Y', 'Direct end-to-end Distance',
        'Mean resulant length', 'WSAD Flag', 'Direction of end-to-end Time', '20 per pairwise velocity', 
        '50 per pairwise velocity', '80 per pairwise velocity', '20 per pairwise acc', '50 per pairwise acc', 
        '80 per pairwise acc', 'Median velocity at last 3pts', 'Largest deviation from end-to-end line', '20 per dev line',
        '50 per dev line', '80 per dev line', 'Average Direction', 'Length of Trajectory', 
        'Ratio dist and length of trajectory', 'Average velocity', 'Median acceleration at first 5 pts',
        'Mid-Stroke pressure', 'Mid-Stroke area covered', 'Mid-Stroke finger orientation', 'Change of finger orientation',
        'Phone orientation']].values

In [6]:
#Choosing the lables for y
y = train['User_ID'].values

In [7]:
#Train and test split
#80% train, 20% test
#splitting the data into test and train dataset

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [8]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
#Training and Prediction
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)         #value for k
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [10]:
#Make prediction on the test data
y_pred = classifier.predict(X_test)

In [15]:
#Evaluating the Algorithm
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 8  0  0 ...  1  0  2]
 [ 0 26  0 ...  0  0  0]
 [ 0  0  7 ...  1  0  0]
 ...
 [ 0  2  1 ...  8  0  0]
 [ 0  0  0 ...  0  6  0]
 [ 0  0  0 ...  0  0  3]]
              precision    recall  f1-score   support

           1       0.89      0.53      0.67        15
           2       0.57      0.84      0.68        31
           3       0.39      0.29      0.33        24
           4       0.57      0.75      0.65        16
           5       0.20      0.27      0.23        11
           6       0.52      0.76      0.62        17
           7       0.54      0.67      0.60        21
           8       0.50      0.62      0.56        24
           9       0.50      0.78      0.61         9
          10       0.25      0.27      0.26        11
          11       0.71      0.71      0.71        14
          12       0.67      0.63      0.65        19
          13       0.33      0.12      0.18        16
          14       0.41      0.75      0.53        24
          15       0.78      0.56

In [ ]:
'''
'macro':
 Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.

'weighted':
 Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label).
 This alters ‘macro’ to account for label imbalance; it can result in an F-score that is not between precision and recall.
'''